# 텍스트 분류 모델 구현

- 데이터 셋 : 200개 한국어 뉴스 기사
- 라 벨 : 정치(0), 경제(1), 사회(2), 생활/문화(3), 세계(4), 기술/IT(5), 연예(6), 스포츠(7)

- 입력 받은 TEXT에 대한 기사 종류 분류 결과 출력

In [1]:
import os 
import pandas as pd
import numpy as np

import spacy                # 형태소 분석기 

from collections import Counter



## [1] 데이터 준비

In [2]:
# 데이터 불러오기 

PATH = '../10_08/NEWS_DATA/'
file_list=  os.listdir(PATH)

file_list

['0', '1', '2', '3', '4', '5', '6', '7']

In [3]:
# 데이터프레임 만들기 

news_data = []

for file_name in file_list:
    for root, dirs, files in os.walk(PATH+file_name):
        print(f'{file_name} 하위 파일 개수 : {len(files)}')
        for file in files:
            with open(root+'/'+file, 'r', encoding='utf-8') as f:
                news_data.append([f.readlines(), file_name])

news_data_df = pd.DataFrame(news_data, columns=['text','label'])

0 하위 파일 개수 : 200
1 하위 파일 개수 : 200
2 하위 파일 개수 : 200
3 하위 파일 개수 : 200
4 하위 파일 개수 : 200
5 하위 파일 개수 : 200
6 하위 파일 개수 : 200
7 하위 파일 개수 : 200


os.walk() 반환 값
- root : dir과 files가 있는 path
- dirs : root 아래에 있는 폴더들
- files : root 아래에 있는 파일들

In [4]:
news_data_df

,text,label
0,"[동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목\t최 부상, 행선지·방...",0
1,"[예결위, 추경 막바지 심사 진통…여야 충돌\t(서울=연합뉴스) 김남권 기자 = 국...",0
2,[외압 논란·항명 사태…산 넘고 물 건넌 권성동 영장 청구\t안미현 검사 외압 폭로...,0
3,"[친문 홍영표, 문빠에 찍혔다…특검 합의에 문자폭탄 공격\t대표적인 친(親)문재인계...",0
4,"[北, 연일 南비난…韓美정상회담 전 경고성 메시지 발신\t南, 맥스선더·태영호 등 ...",0
...,...,...
1595,"[단일팀 추진' 대한카누연맹, 데상트코리아와 5년 후원 협약\t[스포티비뉴스=조형애...",7
1596,"[올림픽 성공 뒷이야기... 서울대, 16일 이희범 평창 조직위원장 초청 특강\t[...",7
1597,[21일 개막 호치민3쿠션월드컵에 149명 참가 확정\t韓선수 46명 참가…1차 예...,7
1598,"[스포츠안전재단, 대축전에 안전필요성 알려\t[스타뉴스 채준 기자]\n, \n, \...",7


In [5]:
news_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1600 non-null   object
 1   label   1600 non-null   object
dtypes: object(2)
memory usage: 25.1+ KB


In [6]:
news_data_df['label']=news_data_df['label'].astype(int)

In [7]:
news_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1600 non-null   object
 1   label   1600 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 18.9+ KB


In [8]:
news_data_df.to_csv('news_data_df.csv')

## [2] Train, Test 데이터 분리

In [9]:
train = news_data_df.sample(frac=0.9, random_state=12) # 90%만 뽑아서 train 
test = news_data_df.drop(train.index) # drop한 나머지 10%를 test

print("Training Data Size :", len(train))
print("Testing Data Size :", len(test))

Training Data Size : 1440
Testing Data Size : 160


In [10]:
train.head(5)

,text,label
1432,"[스포츠경향배, 국산마 ‘가속불패’ 압도적인 기량으로 우승\t20일 경기도 과천 렛...",7
753,[주말 마라톤·차없는거리 행사로 서울 도심 교통통제\t잠실∼성남·광화문∼여의도…대중...,3
677,[몸속 콜레스테롤 줄이는 똑똑한 '지방 섭취법'\t지나친 콜레스테롤은 혈관을 딱딱하...,3
1437,"[여자하키, 아시아 챔피언스트로피 7년 만에 정상 탈환\t(서울=연합뉴스) 고미혜 ...",7
245,"[한진家 5남매 ""해외 상속재산 상속세 납부 시작""\t세무당국 고발로 검찰 수사…탈...",1


In [11]:
test.head(5)

,text,label
17,"[벼랑 끝 전술' 펼치는 北의도는…북미빅딜 전 협상력 올리기\t전문가 ""이럴 때일수...",0
49,"[송영무-문정인, ""宋, 맥스선더에 B-52전폭기 전개 안되게 했다"" 발언 놓고 논...",0
57,[[리얼미터 조사]文대통령 지지율 소폭 하락한 74%…與도 53%\t[아시아경제 유...,0
58,"[""해명해야"" vs ""왜곡됐다""... 남경필 vs 이재명 '욕설 파일 논란' 악화일...",0
82,"[文대통령 ""몰카·데이트폭력은 악성범죄…중대 위법으로 다뤄야""\t[머니투데이 최경민...",0


## [3] 데이터 전처리
### 불용어 및 구두점 제거

In [12]:
# 언어 모델 모델 설정 
LANG_MODEL = 'ko_core_news_lg'

In [13]:
# 한국어 분석기 생성 
nlp = spacy.load(LANG_MODEL)

In [14]:
# 추가할 불용어 정의
new_stop_words = ["\t", "\n"]

# 불용어 목록에 추가
for word in new_stop_words:
    nlp.Defaults.stop_words.add(word)
    nlp.vocab[word].is_stop = True

# 확인
print(nlp.Defaults.stop_words)  

{'어떻', '못하', '좀', '그런', '위하', '있', '더', '어떤', '놓', '그', '보', '\t', '살', '좋', '그리고', '그것', '다른', '안', '번', '알', '원', '나', '이', '않', '주', '없', '아니', '되', '년', '말하', '수', '때', '때문', '많', '\n', '잘', '싶', '한', '같', '오', '그렇', '그러나', '크', '시키', '두', '받', '지', '하나', '데', '등', '말', '하', '점', '일', '또', '그러', '이렇', '것', '들', '가'}


In [15]:
def remove(data, list):

    for text in data['text']:
        sentence = []
        doc = nlp(text[0])

        for token in doc:
            if (not token.is_punct) and (not token.is_stop):
                sentence.append(str(token))
                # 구두점과 불용어 아닌 것들은 다 넣음
        list.append(sentence)

    return list
    

In [16]:
train_list = []
remove(train, train_list)

train_list[:100]

[['스포츠경향배',
  '국산마',
  '가속불패',
  '압도적인',
  '기량으로',
  '우승',
  '20일',
  '경기도',
  '과천',
  '렛츠런파크',
  '서울에서',
  '펼쳐진',
  '제9회',
  '스포츠경향배',
  '1등급',
  '1400',
  'm',
  '연령오픈',
  '핸디캡',
  '경주에서',
  '국산마',
  '가속불패',
  '압도적인',
  '기량으로',
  '우승했다',
  '1분',
  '25초',
  '6의기록으로',
  '결승선을',
  '통과했다',
  '\xa0\n'],
 ['주말',
  '마라톤',
  '차없는거리',
  '행사로',
  '서울',
  '도심',
  '교통통제',
  '잠실∼성남',
  '광화문∼여의도',
  '대중교통',
  '이용',
  '교통정보',
  '확인',
  '바람직'],
 ['몸속',
  '콜레스테롤',
  '줄이는',
  '똑똑한',
  '지방',
  '섭취법',
  '지나친',
  '콜레스테롤은',
  '혈관을',
  '딱딱하게',
  '하고',
  '혈관을',
  '막는',
  '혈전',
  '피떡',
  '을',
  '만들어',
  '혈관',
  '질환의',
  '주범으로',
  '작용한다',
  '때문에',
  '콜레스테롤이',
  '많이',
  '들었다고',
  '알려진',
  '달걀노른자',
  '등의',
  '특정',
  '식품들을',
  '가려먹는',
  '사람이',
  '많다',
  '하지만',
  '특정',
  '음식을',
  '피하는',
  '것뿐',
  '아니라',
  '다양한',
  '지방',
  '섭취량을',
  '조절하는',
  '것도',
  '혈중',
  '콜레스테롤을',
  '줄이는',
  '도움이',
  '된다'],
 ['여자하키',
  '아시아',
  '챔피언스트로피',
  '7년',
  '만에',
  '정상',
  '탈환',
  '서울',
  '=',
  '연합뉴스',
  '고미혜',
  '기자',
  '=',
  '여자',


In [17]:
test_list = []
remove(test, test_list)

test_list[:10]

[['벼랑',
  '끝',
  '전술',
  '펼치는',
  '北의도는',
  '북미빅딜',
  '전',
  '협상력',
  '올리기',
  '전문가',
  '이럴',
  '때일수록',
  '정상',
  '간',
  '핫라인',
  '가동해야'],
 ['송영무',
  '문정인',
  '宋',
  '맥스선더에',
  'B-52전폭기',
  '전개',
  '안되게',
  '했다',
  '발언',
  '놓고',
  '논란',
  '서울',
  '=',
  '뉴시스',
  '김성진',
  '기자',
  '=',
  '북한이',
  '미',
  '공군',
  '연합훈련인',
  '맥스선더',
  'Max',
  'Thunder',
  '훈련을',
  '이유로',
  '남북',
  '고위급',
  '회담',
  '중지를',
  '선언한',
  '가운데',
  '미국',
  '전략폭격기',
  'B-52의',
  '맥스선더',
  '훈련',
  '참가',
  '여부와',
  '관련해',
  '송영무',
  '국방부',
  '장관과',
  '문정인',
  '대통령',
  '외교안보특보',
  '간에',
  '말이',
  '엇갈리면서',
  '논란이',
  '일고',
  '있다'],
 ['리얼미터',
  '조사]文대통령',
  '지지율',
  '소폭',
  '하락한',
  '74%',
  '與도',
  '53',
  '아시아경제',
  '유제훈',
  '기자',
  '문재인',
  '대통령의',
  '국정수행',
  '지지율이',
  '전주',
  '대비',
  '소폭',
  '하락한',
  '74%를',
  '기록했다',
  '집권여당인',
  '더불어민주당의',
  '지지율도',
  '53%로',
  '조사됐다'],
 ['해명해야',
  'vs',
  '왜곡됐다',
  '남경필',
  'vs',
  '이재명',
  '욕설',
  '파일',
  '논란',
  '악화일로',
  '남경필',
  '후보',
  '17일',
  '긴급기자회견',
  '열고',
  '이재명',
  

## [4] 단어사전 구축

In [18]:
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

In [19]:
train_vocab = build_vocab(corpus = train_list, n_vocab=5000, special_tokens=["<pad>", "<unk>"]) 
test_vocab = build_vocab(corpus = test_list, n_vocab=5000, special_tokens=["<pad>", "<unk>"]) 

id_to_token_train = {idx: token for idx, token in enumerate(train_vocab)}

In [20]:
token_to_id_train = {token: idx for idx, token in enumerate(train_vocab)}

In [21]:
len(train_vocab), len(test_vocab)

(5002, 2522)

In [22]:
id_to_token_train

{0: '<pad>',
 1: '<unk>',
 2: '=',
 3: '\xa0',
 4: '기자',
 5: '\xa0\n',
 6: '서울',
 7: '종합',
 8: '있다',
 9: '연합뉴스',
 10: '앵커',
 11: '만에',
 12: '배우',
 13: '한국',
 14: '중',
 15: '<',
 16: '>',
 17: '전',
 18: '단독',
 19: '있는',
 20: '2018',
 21: '드루킹',
 22: '뉴시스',
 23: '에',
 24: '첫',
 25: '위해',
 26: '밝혔다',
 27: '트럼프',
 28: '北',
 29: '것으로',
 30: '미국',
 31: '머니투데이',
 32: '논란',
 33: '가운데',
 34: 'OSEN',
 35: '지난',
 36: '대한',
 37: '특검',
 38: '서울신문',
 39: '경찰',
 40: '~',
 41: '한겨레',
 42: '의',
 43: '특파원',
 44: '듯',
 45: '최근',
 46: '5',
 47: '를',
 48: '일부',
 49: '큰',
 50: '후',
 51: '김정은',
 52: '수사',
 53: '정부',
 54: '뉴스1',
 55: '미',
 56: '국내',
 57: '30대',
 58: '우승',
 59: '및',
 60: '오는',
 61: '오후',
 62: '16일',
 63: '없는',
 64: '나우뉴스',
 65: '혐의',
 66: '부산',
 67: '함께',
 68: '中',
 69: '현지시간',
 70: '위한',
 71: '대해',
 72: '18일',
 73: '중국',
 74: '을',
 75: '전국',
 76: '관련',
 77: '여야',
 78: '새',
 79: 'vs',
 80: '스포츠서울',
 81: '공개',
 82: '가능성',
 83: '김성태',
 84: '여자',
 85: '개최',
 86: '발표',
 87: '여성',
 88: '19일',
 89: 

## [5] 단어 인코딩 및 패딩

In [23]:
def max_length(data):
    len_list = []
    for tokenList in data:
        len_list.append(len(tokenList))

    MAX_LENGTH = max(len_list)
    return MAX_LENGTH

In [24]:
print(f'train max_length : {max_length(train_list)} , test max_length : {max_length(test_list)}')

train max_length : 435 , test max_length : 95


In [25]:
def pad_sequences(sequences, max_length, pad_value):
    result = list() 
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence =  sequence + [pad_value] * pad_length
        result.append(padded_sequence)

    return np.asarray(result)


unk_id = token_to_id_train["<unk>"]
train_ids = [[token_to_id_train.get(token, unk_id) for token in text] for text in train_list]
test_ids = [[token_to_id_train.get(token, unk_id) for token in text] for text in test_list]

## 패딩 
max_length = 435
pad_id = token_to_id_train["<pad>"]
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

In [26]:
print(train_ids[0])
print(test_ids[0])

[ 332  409  653  654  918   58  158  655 4289  235  656 2109 4290  332
  919  920  144 4291 4292 4293  409  653  654  918 4294  921 4295 4296
 1346 2110    5    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

## [6] 모델 학습 준비

In [27]:
type(train.label.values)

numpy.ndarray

In [28]:
train.label.values

array([7, 3, 3, ..., 2, 7, 2])

In [29]:
# 데이터로더 적용 

import torch 
from torch.utils.data import TensorDataset, DataLoader

train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

## object 타입은 tensor로 변환이 불가하므로 label을 숫자형으로 변경 
train_labels = torch.tensor(train.label.values, dtype = torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

## 모델 학습 과정 시 각 step 마다 데이터를 batch size 크기로 분할하여 넣어 효과적이고 효율적인 학습 진행하도록 
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [30]:
# 문장 분류 모델 

from torch import nn 

class SentenceClassifier(nn.Module):
    def __init__(
        self, 
        n_vocab, 
        hidden_dim,
        embedding_dim, 
        n_layers,
        dropout = 0.5,
        bidirectional = True,
        model_type = "lstm"
    ):

        super().__init__()

        self.embedding = nn.Embedding( # 인스턴스 생성 
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )

        if model_type == "rnn":
            self.model = nn.RNN(
                input_size = embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size = embedding_dim,
                hidden_size = hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2,1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)


    def forward(self, inputs): # 순방향 메서드
        embeddings = self.embedding(inputs) # 입력받은 정수 인코딩을 임베딩 계층에 통과시켜 임베딩 값을 얻음
        output, _ = self.model(embeddings) # 얻음 임베딩 값을 모델에 입력하여 출력값 얻음 
        last_output = output[:,-1,:] 
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output) # 마지막 값만 추출하여 분류기 계층에 전달 
        return logits



In [31]:
# 손실함수와 최적화 함수 정의 

from torch import optim

n_vocab = len(token_to_id_train)
hidden_dim = 64             # 은닉 상태 크기 64
embedding_dim = 64         # embedding 벡터 크기 64
n_layers = 4                # 신경망을 2개의 층으로 구성 

device = 'cuda' if torch.cuda.is_available() else "cpu"
classifier = SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
).to(device)

criterion = nn.CrossEntropyLoss().to(device)   

## RMSprop : 모든 기울기를 누적하지 않고, 지수 가중 이동 평균(EWMA)을 사용해 학습률 조절 
## - 기울기 제곱 값의 평균값이 작아지면 학습률 증가, 반대일 경우 학습률을 감소시켜 불필요한 지역 최솟값에 빠지는 것 방지 
## - 기울기의 크기가 큰 경우에는 빠른 수렴을 보이며 작은 경우에는 더 작은 학습률을 유지시켜 더 안정적으로 최적화 수행 
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

In [32]:
# 모델 학습 및 테스트 
# - 모델 학습 중간에 학습이 잘 이뤄지고 있는지 확인하기 위해 일정 배치 학습 후 테스트 데이터세트로 손실값 확인 

def train(model, datasets, criterion, optimizer, device, interval):
    model.train() # train 모드 설정 
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1) # unsqueeze : 지정한 자리에 size가 1인 빈 공간을 채워주면서 차원을 확장

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f'Train Loss {step} : {np.mean(losses)}')


def test(model, datasets, criterion, device):
    model.eval() # 검증 모드 설정 
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels= labels.to(device).unsqueeze(1)
        
        logits = model(input_ids)
        loss= criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.softmax(logits, dim=1) > .5 
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )

    print(f'Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}')


epochs = 100
interval = 500

## 위의 코드에서 정의한 criterion, optimizer 사용 
for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device) # 검증은 test 데이터셋으로 진행 

Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy : 0.075
Train Loss 0 : 0.0
Val Loss : 0.0, Val Accuracy 

....???????????